In [1]:
import copy
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'RM_baseline')))

from model_copy import DoraNet
from util import *
from dataset import DoraSet, DoraSetComb

In [2]:
epochs = 500  # total epochs
local_epochs = 5 # local epochs of each user at an iteration
saveLossInterval = 1  # intervals to save loss
saveModelInterval = 10  # intervals to save model
batchSize = 500  # batchsize for training and evaluation
num_users = 90   # total users
num_activate_users = 5
lr = 3e-4  # learning rate
cudaIdx = "cuda:0"  # GPU card index
device = torch.device(cudaIdx if torch.cuda.is_available() else "cpu")
num_workers = 0  # workers for dataloader
evaluation = False  # evaluation only if True

In [ ]:
folder = "./data/train/"
userlist = list(range(1, 91))
# userlist = list(range(1, 11))
datalist = []

count = 0
for i in userlist:
    clientId = i
    data_i = np.reshape(np.load(folder+f'user_{clientId:02d}.npy'),(-1,6))
    datalist.append(data_i)
    if clientId % 10 == 0:
        print(count)
        count = 0
    else:
        count += data_i.shape[0]

# data = np.vstack(datalist)
# plt.figure(figsize=(16, 12))
# plt.scatter(data[:, 0], data[:, 1], s=6, c=data[:,5])
# plt.colorbar()

In [ ]:
folder = "./data/train/"
userlist = list(range(1, 91))
# userlist = list(range(1, 11))
datalist = []

bs = 1
for i in userlist:
    clientId = i
    data_i = np.reshape(np.load(folder+f'user_{clientId:02d}.npy'),(-1,6))
    # data_i[:, 2:] = 10 ** (data_i[:, 2:] / 10)
    # datalist.append(data_i[data_i[:, 1 + bs] != 0, :])
    datalist.append(data_i)

data = np.vstack(datalist)
print(data[:, 0].min(), data[:, 0].max(), data[:, 1].min(), data[:, 1].max())
print(data[:, 2:].min(), data[:, 2:][data[:, 2:] != 0].max())
# plt.figure(figsize=(16, 12))
# plt.scatter(data[:, 0], data[:, 1], s=6, c=data[:, 1 + bs])
# plt.colorbar()

# userlist = list(range(1, 91))
# datalist = []

# bs = 1
# for i in userlist:
#     clientId = i
#     data_i = np.reshape(np.load(folder+f'user_{clientId:02d}.npy'),(-1,6))
#     # data_i[:, 2:] = 10 ** (data_i[:, 2:] / 10)
#     datalist.append(data_i[data_i[:, 1 + bs] != 0, :])

# data = np.vstack(datalist)
# plt.figure(figsize=(16, 12))
# plt.scatter(data[:, 0], data[:, 1], s=6, c=data[:, 1 + bs])
# plt.colorbar()

# idx 2, bs 1, bottomleft
# idx 3, bs 2, upperleft
# idx 4, bs 3, bottomright
# idx 5, bs 4, upperright

In [3]:
model = DoraNet()
model.load_state_dict(torch.load('../tarfile/models/model500.pth'))
model.to(device)
train_dataset_path = 'data/train/'
valid_datasets = []
for i in range(1, num_users + 1):
    all_dataset = DoraSet(train_dataset_path, set='train', clientId=i)
    # train_size = int(0.99 * len(all_dataset))
    # valid_size = len(all_dataset) - train_size
    # train_dataset, valid_dataset = torch.utils.data.random_split(all_dataset, [train_size, valid_size])
    # train_datasets.append(train_dataset)
    # valid_datasets.append(valid_dataset)
    valid_datasets.append(all_dataset)

valid_data_comb = DoraSetComb(valid_datasets)
valid_loader = torch.utils.data.DataLoader(valid_data_comb, 500, shuffle=False, num_workers=num_workers)

def valid(data_loader, model):
    with torch.no_grad():
        model.eval()
        # losses = Recoder()
        # scores = Recoder()
        count = 0
        accuracy = 0
        for i, (pos, pathloss) in enumerate(data_loader):
            pos = pos.float().to(device)
            pathloss = pathloss.float().to(device)
            # p_pathloss = model(pos)
            # loss = torch.mean(torch.abs(p_pathloss[pathloss != 0] - pathloss[pathloss != 0])) ## unit in dB
            # tmp1 = torch.sum(torch.abs(10 ** (0.1 * p_pathloss[pathloss != 0]) - 10 ** (0.1 * pathloss[pathloss != 0])) ** 2)
            # tmp2 = torch.sum(torch.abs(10 ** (0.1 * pathloss[pathloss != 0])) ** 2)
            # score = tmp1 / tmp2
            # if score>1:
            #     score=torch.tensor([1])
            # losses.update(loss.item(), len(pos))
            # scores.update(score.item(), len(pos))
            pred = model(pos)
            mask = torch.where(pathloss != 0., 1., 0.).float().to(device)
            accuracy += torch.sum(pred == mask).item()
            count += mask.numel()
    # return -10 * np.log10(scores.avg())
    return accuracy / count * 100

valid(valid_loader, model)

97.29958722845134